In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

In [52]:
soup_prueba.find('div',{'class':"b-leading-data-property u-flex-wrap"}) != None

False

In [60]:
dict_divs_info = {}
def scrap_div_info(soup_cada_casa, o_ = 0): #Para cada casa, sacame esta info.
    direccion_ = soup_cada_casa.find('div',{'class': 'b-ubication'}).find_all('p')[0].text
    anunciante_ = soup_cada_casa.find('div',{'class':'b-name-agent'}).find_all('p')[0].text
    florito_ = soup_cada_casa.find('div',{'class':'b-section-content js-property-services'}).findNext('div').find('p').text
    fecha_pub = soup_cada_casa.find('div',{'class':'b-view-code'}).findNext('span').text
    precio = soup_cada_casa.find('p',{'class':'e-totalPrice'}).text
    #visitas_ = soup_cada_casa.find('div',{'class':'b-view-code'}).find_all('span')[1].text
    global dict_divs_info
    dict_SUB_divs_info = {'Direccion':direccion_,'Anunciante':anunciante_,'Descripcion_':florito_,
                          'Fecha_pub':fecha_pub, 'Precio': precio}
    dict_divs_info['Casa'+str(o_)] = dict_SUB_divs_info
    #fill_nan_vals_dict(dict_divs_info)
    return dict_divs_info

In [61]:
url = 'https://urbania.pe/inmueble/proyecto-condominio-club-casas-del-valle-lima-carabayllo-enacorp-sa-574'
response = requests.get(url)
soup_prueba = BeautifulSoup(response.text,'html.parser')

In [62]:
url_sin = 'https://urbania.pe/inmueble/venta-de-casa-en-san-isidro-lima-4-dormitorios-4156378'
response = requests.get(url_sin)
soup_prueba2 = BeautifulSoup(response.text,'html.parser')

In [63]:
scrap_div_info(soup_prueba)

{'Casa0': {'Direccion': 'Av. San Juan s/n - Alt. Km. 30 Pan. Norte - Carabayllo Carabayllo, Lima, Lima',
  'Anunciante': 'ENACORP S.A.',
  'Descripcion_': ' ENACORP presenta Condominio Club Casas del Valle, un Moderno Condominio Residencial totalmente cerrado, que cuenta con Casas de 02 Pisos de 71.5 m2, con Sala Comedor, 03 Dormitorios, Cocina, LavanderÃ\xada, JardÃ\xadn, y 2 baÃ±os, con opciÃ³n a ampliaciÃ³n a un piso 03, cuenta con Piscinas, Gym, SalÃ³n de Juegos y de reuniones. Por ello te invitamos a disfrutar de conocer La Casa SoÃ±ada, dentro de un Condominio vigilado y seguro las 24 horas del dÃ\xada pensando en la Familia.Â\xa0  Consulta por Nuestras Promociones y empieza a vivir como soÃ±aste en TU CASA PROPIA. Contacte a nuestros asesores para mayor informaciÃ³n:  Caseta Ventas : \xa0ver datos\xa0 - \xa0ver datos\xa0 MÃ³viles: \xa0ver datos\xa0, \xa0ver datos\xa0, \xa0ver datos\xa0  Trabajamos con Scotiabank  Separa tu Casa con S/. 2000  Visita nuestra Web: \xa0ver datos\xa0

In [114]:
def merge(a, b, path=None):
    "merges b into a"
    if path is None: path = []
    for key in b:
        if key in a:
            if isinstance(a[key], dict) and isinstance(b[key], dict):
                merge(a[key], b[key], path + [str(key)])
            elif a[key] == b[key]:
                pass # same leaf value
            else:
                raise Exception('Conflict at %s' % '.'.join(path + [str(key)]))
        else:
            a[key] = b[key]
    return a

def fill_nan_vals_dict(dictionary):
    for dict_per_house in dictionary.values():
        if dict_per_house == {}:
            dict_per_house['NoTieneAlgunDatoExtra'] = 'NingunValor'
        else:
            pass

dict_divs_info = {}
def scrap_div_info(soup_cada_casa, o_ = 0): #Para cada casa, sacame esta info.
    direccion_ = soup_cada_casa.find('div',{'class': 'b-ubication'}).find_all('p')[0].text
    if soup_cada_casa.find('div',{'class':'b-name-agent'}) == None:
        anunciante_ = 'No disponible'
    else:
        anunciante_ = soup_cada_casa.find('div',{'class':'b-name-agent'}).find_all('p')[0].text
    florito_ = soup_cada_casa.find('div',{'class':'b-section-content js-property-services'}).findNext('div').find('p').text
    fecha_pub = soup_cada_casa.find('div',{'class':'b-view-code'}).findNext('span').text
    precio = soup_cada_casa.find('p',{'class':'e-totalPrice'}).text
    #visitas_ = soup_cada_casa.find('div',{'class':'b-view-code'}).find_all('span')[1].text
    global dict_divs_info
    dict_SUB_divs_info = {'Direccion':direccion_,'Anunciante':anunciante_,'Descripcion_':florito_,
                          'Fecha_pub':fecha_pub, 'Precio': precio}
    dict_divs_info['Casa'+str(o_)] = dict_SUB_divs_info
    #fill_nan_vals_dict(dict_divs_info)
    return dict_divs_info

def obtain_features_to_dict(soup_cada_casa,n_=0 ):  #Debería poner el n_ afuera de la función para iterar luego?
    obj_features_dicc = soup_cada_casa.find('section',{'class':'feature'})
    if obj_features_dicc == None:
        casa_interna_iterada = {}
    else:
        obj_features_dicc = obj_features_dicc.find_all('section') #Para cada casa selecciono su section
        n_ += 1 #A lo mejor es importante el número de casa
        casa_interna_iterada = {} #dict_3ro
        for section in obj_features_dicc:
            feature = section.find('h2').text #Este va a ser mi key de los servicios (features) que va a tener la data.
            sub_feature = [lisst.text for lisst in section.find_all('li',{'class':'b-section-item'})] #Serán los values 
            ##Entonces, segùn lo de arriba, para cada feature habrá una lista.
            casa_interna_iterada[feature] = sub_feature #Agrega cada feature con sus respectiva lista al diccionario
    global dict_feature_all_casas #dict_2do
    dict_feature_all_casas['Casa'+str(n_)] = casa_interna_iterada #Mi diccionario de diccionarios {'Casa1:{'Servicios': [],...}
    fill_nan_vals_dict(dict_feature_all_casas)
    return dict_feature_all_casas #Me retorna el diccionario de diccionarios

dict_property_details = {}
def dict_property_info(soup_cada_casa, u_=0):
    global dict_property_details #A un dict de afuera
    obj_dicc2 = soup_cada_casa.find('div',{'class':'b-property-details u-flex-wrap'}).find_all(
        'div',{'class':"b-leading-data-service"}) #Busco la lista para iterar
    dict_casa_iter2 = {} #Creo un dict interno
    for iter_ in obj_dicc2:#itero
        key1 = iter_.find_all('p')[0].text #Saco mi key
        value1 = iter_.find_all('p')[1].text #Saco mi value
        dict_casa_iter2[key1] = value1 #Lo agrego al diccionario interno
    fill_nan_vals_dict(dict_casa_iter2)
    dict_property_details['Casa'+str(u_)] = dict_casa_iter2
##Funcion General de scraping..............................

### Esta es la función principal.
#soup_list = [soup1,soup2,soup3]
dict_all_casas = {} #dict_1ro
dict_feature_all_casas = {} #dict_2do
dict_property_details = {}

def my_urbania_scrapper(all_internal_link):
    global dict_all_casas #dict_all_casas
    global dict_property_details
    global dict_feature_all_casas #Creo q no es necesario hacerlo global? #dict_all_casas_2
                #Para cada cuadro de información de cada casa
    for i,internal_link in tqdm(enumerate(all_internal_link)):
        response = requests.get(internal_link)
        soup_cada_casa = BeautifulSoup(response.text,"html.parser") #con esto ya no necesitaré listas.
        try:
            obtain_features_to_dict(soup_cada_casa,n_=i) ##recuerda que esta f(x) lo bota en dict_feature_all_casas##PabloBenavides
        except:
            pass
        try: 
            dict_casas_info(soup_cada_casa,i_=i) ##Recuerda que esta f(x) lo bota en dict_all_casas.
        except AttributeError: 
            pass 
        try:
            dict_property_info(soup_cada_casa, u_= i)
        except AttributeError:
            pass
        try:
            scrap_div_info(soup_cada_casa,o_=i) #Me lo bota en dict_divs_info
        except:
            pass
    prefinal_dict = merge(dict_all_casas,dict_feature_all_casas)
    final_dict = merge(prefinal_dict,dict_divs_info)
    final_final_dict = merge(final_dict, dict_property_details)
    final_df = pd.DataFrame.from_dict(final_final_dict,orient='index')
    final_df.to_csv('database_urbania666.csv')


In [115]:
links_prueba = ['https://urbania.pe/inmueble/proyecto-condominio-club-casas-del-valle-lima-carabayllo-enacorp-sa-574',
               'https://urbania.pe/inmueble/venta-de-casa-en-san-isidro-lima-4-dormitorios-4156378',
               'https://urbania.pe/inmueble/venta-de-casa-en-san-borja-lima-3-dormitorios-4379311',
               'https://urbania.pe/inmueble/venta-de-casa-en-san-miguel-lima-5-a-mas-dormitorios-4349263']

In [119]:
my_urbania_scrapper(links_prueba)

4it [00:04,  1.07s/it]


In [105]:
url3 = 'https://urbania.pe/inmueble/venta-de-casa-en-san-borja-lima-3-dormitorios-4379311'
response = requests.get(url3)
soup3 = BeautifulSoup(response.text,'html.parser')

In [121]:
dict_feature_all_casas = {}
obtain_features_to_dict(soup3)
dict_feature_all_casas

{'Casa0': {'Ambientes': ['JardÃ\xadn Interno',
   'Patio',
   'Walking Closet',
   'BaÃ±o de servicio',
   'Cuarto de servicio',
   'LavanderÃ\xada',
   'BaÃ±o de visitas',
   'Cocina',
   'Comedor',
   'Comedor diario',
   'Sala',
   'Sala de estar',
   'Terraza'],
  'Adicionales': ['Parrilla', 'Chimenea'],
  'Ã\x81reas comunes': ['Hall de ingreso', 'JardÃ\xadn', 'Piscina']}}

In [118]:
def obtain_features_to_dict(soup_cada_casa,n_=0 ):  #Debería poner el n_ afuera de la función para iterar luego?
    obj_features_dicc = soup_cada_casa.find('section',{'class':'feature'})
    if obj_features_dicc == None:
        casa_interna_iterada = {'NoTieneAlgunDatoExtra':'NingunValor'}
    else:
        obj_features_dicc = obj_features_dicc.find_all('section') #Para cada casa selecciono su section
        casa_interna_iterada = {} #dict_3ro
        for section in obj_features_dicc:
            feature = section.find('h2').text #Este va a ser mi key de los servicios (features) que va a tener la data.
            sub_feature = [lisst.text for lisst in section.find_all('li',{'class':'b-section-item'})] #Serán los values 
            ##Entonces, segùn lo de arriba, para cada feature habrá una lista.
            casa_interna_iterada[feature] = sub_feature #Agrega cada feature con sus respectiva lista al diccionario
    global dict_feature_all_casas #dict_2do
    dict_feature_all_casas['Casa'+str(n_)] = casa_interna_iterada #Mi diccionario de diccionarios {'Casa1:{'Servicios': [],...}
    fill_nan_vals_dict(dict_feature_all_casas)
    return dict_feature_all_casas #Me retorna el diccionario de diccionarios

In [120]:
df = pd.read_csv('database_urbania666.csv')
df

,Unnamed: 0,Tipo,Antiguedad,Dormitorios,BaÃ±os,Ãrea Total,Ãrea Construida,Area de Terreno,Cocheras,UbicaciÃ³n,UrbanizaciÃ³n,NoTieneAlgunDatoExtra,Direccion,Anunciante,Descripcion_,Fecha_pub,Precio,DisposiciÃ³n,Estado de Inmueble,Luminosidad,Cantidad de pisos,PortÃ³n Cochera,Tipo de Cochera,Fondo,Frente,Pisos construibles,Referencia,Ambientes,Adicionales,Ãreas comunes,Servicios,Generales
0,Casa0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NingunValor,Av. San Juan s/n - Alt. Km. 30 Pan. Norte - Ca...,ENACORP S.A.,ENACORP presenta Condominio Club Casas del Va...,Publicado el 27.07.19,"S/ 175,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Casa1,Casa,38 aÃ±os,4,4.0,360.00 m2,330.00 m2,360.00 m2,3.0,San Isidro-Lima,Corpac,NingunValor,"Ricardo Angulo, CÃ³rpac San Isidro, Corpac, Li...",ARB Asesores Inmobiliarios,"Linda casa, A.T. 360 m2 (12 x 30), excelente ...",Publicado el 23.08.19,"US$720,000",Frente,Bueno,Luminoso,2.0,AutomÃ¡tico,Paralelas,30.0,12.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Casa2,Casa,18 aÃ±os,3,3.0,720.00 m2,300.00 m2,NaN,2.0,San Borja-Lima,Chacarilla,NaN,"Espalda Del Boccata San Borja, Chacarilla, Lim...",No disponible,"Casa de bajos, con techos a 2 aguas de madera...",Publicado el 22.08.19,"US$1,250,000",NaN,Muy bueno,NaN,3.0,NaN,NaN,NaN,NaN,NaN,ESPALDA DEL BOCCATA,"['JardÃ\xadn Interno', 'Patio', 'Walking Close...","['Parrilla', 'Chimenea']","['Hall de ingreso', 'JardÃ\xadn', 'Piscina']",NaN,NaN
3,Casa3,Casa,50 aÃ±os,5 a mÃ¡s,3.0,328.05 m2,226.45 m2,180.00 m2,2.0,San Miguel-Lima,Las Leyendas,NaN,"Calle Huandoy, A Espalda De Parque, Cerca A Pr...",BRASAN HOUSE PERÃ,"Vendo casa de 3 pisos como terreno, parÃ¡metr...",Publicado el 21.08.19,"US$260,000",Frente,A Remodelar,NaN,3.0,NaN,Paralelas,20.0,9.0,5.0,"Av. Precursores, Av. Faucet, Av. Venezuela","['Cocina', 'Comedor', 'Comedor diario', 'Dormi...",NaN,NaN,"['Agua', 'GuardianÃ\xada']","['Centros comerciales cercanos', 'Colegios cer..."
4,Casa4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Cocina', 'Comedor', 'Comedor diario', 'Dormi...",NaN,NaN,"['Agua', 'GuardianÃ\xada']","['Centros comerciales cercanos', 'Colegios cer..."


In [113]:
obtain_features_to_dict(soup_prueba,n_=0)

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [32]:
soup_prueba2.find_all('div',{'class','b-feature-section'})

[]